<a href="https://colab.research.google.com/github/LKXoro/BEDU-Rstudio-postworks/blob/main/M3_Postwork7/Postwork_07_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Postwork 07: Transformación, filtración y ordenamiento de datos 
Antes de comenzar en caso de no tener nuestro confiable archivo csv a la mano, o bien debido a que la API está en constante actualización podemos recurrir a una versión mejorada de la función para peticiones de los **enfrentamientos**:

In [330]:
import requests #Importamos los paquete necesarios
import pandas as pd 
import math as mt
 
def PandaScore_get_matches(juego): #Definimos nuestra función
    if isinstance(juego, str): #Nos aseguramos que los argumentos sean de tipo str
        informacion = 'matches'
        registros = 100 #Establecemos el número de registros por página
        token = 'V3U1fnnWozJi3bbhZfrxdiAmN4-XWnX_4g6i_6iauwwh7w-V8Ns' #Token de acceso
        url = f"https://api.pandascore.co/{juego}/{informacion}?page[size]={registros}&token={token}" #URL con los parámetros deseados
        response = requests.get(url) #Realizamos la petición
        if response.status_code == 200: #Si todo salió bien continuamos
            Iteraciones = mt.ceil(float(response.headers['X-Total'])/registros) #Calculamos el número de páginas en base a todos los registros que hay en la API
            #Al dividir el número total de registros entre los registros solicitados obtenemos un aproximado, para asegurar que tomaremos todos, redondeamos al entero más alto
            for i in range(1,Iteraciones+1): #Ciclo for para obtener todos los registros de cada página
                url = f"https://api.pandascore.co/{juego}/{informacion}?page[size]={registros}&page[number]={i}&token=V3U1fnnWozJi3bbhZfrxdiAmN4-XWnX_4g6i_6iauwwh7w-V8Ns"
                #URL el parámetro de página deseada
                response = requests.get(url) #Realizamos la petición correspondiente
                data = response.json() #Transformamos la respuesta 
                if i == 1: #En la primer iteración se crea un Dataframe
                    df_matches = pd.DataFrame.from_dict(data)
                else: #En el resto de iteraciones se van añadiendo los nuevos registros
                    df_matches = df_matches.append(pd.DataFrame.from_dict(data),ignore_index=True) 
            
            ########### Limpieza de datos
            campos_no_deseados = ['videogame', 'slug', 'tournament', 'league', 'serie', 'streams', 'modified_at', 'games',
                      'game_advantage', 'scheduled_at', 'original_scheduled_at', 'detailed_stats','live',
                      'match_type', 'videogame_version', 'live_embed_url', 'id'] 

            df_matches_temp = df_matches.drop(columns=campos_no_deseados) #Limpiamos campos que no nos son de utilidad
            df_matches_finished = df_matches_temp[ df_matches_temp['status'] == 'finished' ] #Dejamos solo los enfrentamientos que han terminado
            df_matches_finished_clean = df_matches_finished.dropna(subset=['begin_at', 'end_at']) #Eliminamos algunos valores nulos de las fechas
            df_matches_ordenado = df_matches_finished_clean[['name','opponents','results','forfeit','draw', #Reorganizamos los campos
                                                      'number_of_games','winner','winner_id','begin_at','end_at', 
                                                      'rescheduled','official_stream_url','league_id', 'serie_id', 'tournament_id']]
            df_matches_ordenado = df_matches_ordenado.reset_index(drop=True) #Reseteamos el índice
            nombres_nuevos = {'name' : 'match_name',
                             'begin_at' : 'begin_date',
                             'end_at' : 'end_date',
                             'official_stream_url' : 'official_stream_channel_url'}                       
            df_matches_ordenado.rename(columns=nombres_nuevos, inplace=True) #Renombramos columnas                                      
            return df_matches_ordenado # Finalmente se devuelve el Dataframe con todos los registros limpios y ordenados
        else: #Si la petición no se ejecutó correctamente mandamos un mensaje con el error devuelto
            return 'Error de petición: '+str(response.status_code)  
    else: #Si los argumentos no son de tipo str regresamos un mensaje
        return 'Por favor coloque un argumento válido en formato string.'

Le indicamos que deseamos los enfrentamientos del juego League of Legends:


In [422]:
lol_matches = PandaScore_get_matches('lol')

Observamos el Dataframe que nos devuelve la función:

In [423]:
lol_matches.head()

,match_name,opponents,results,forfeit,draw,number_of_games,winner,winner_id,begin_date,end_date,rescheduled,official_stream_channel_url,league_id,serie_id,tournament_id
0,ALF vs BJD,"[{'opponent': {'acronym': 'ALF', 'id': 124408,...","[{'score': 0, 'team_id': 124408}, {'score': 1,...",False,False,1,"{'acronym': 'BJD', 'id': 126861, 'image_url': ...",126861.0,2021-03-06T09:54:42Z,2021-03-06T10:28:54Z,False,https://www.twitch.tv/lolpacific,4288,3337,5483
1,Lower Bracket Round 2 Match 1: CGG vs GGE,"[{'opponent': {'acronym': 'CGG', 'id': 127494,...","[{'score': 1, 'team_id': 127494}, {'score': 3,...",False,False,5,"{'acronym': 'GGE', 'id': 128405, 'image_url': ...",128405.0,2021-03-08T18:07:48Z,2021-03-08T21:37:26Z,False,https://www.twitch.tv/pg_esports,4405,3291,5579
2,Round 1 match 1: KVM vs ION,"[{'opponent': {'acronym': 'KVM', 'id': 127475,...","[{'score': 3, 'team_id': 127475}, {'score': 0,...",False,False,5,"{'acronym': 'KVM', 'id': 127475, 'image_url': ...",127475.0,2021-03-08T18:04:03Z,2021-03-08T20:46:01Z,False,https://www.twitch.tv/esportsprimelol,4401,3248,5609
3,Winners bracket round 1 match 2: ESB vs SIN,"[{'opponent': {'acronym': 'ESB', 'id': 3344, '...","[{'score': 2, 'team_id': 3344}, {'score': 3, '...",False,False,5,"{'acronym': 'SIN', 'id': 127586, 'image_url': ...",127586.0,2021-03-07T17:53:43Z,2021-03-07T22:22:51Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5602
4,Winners bracket round 1 match 1: IG vs VIK,"[{'opponent': {'acronym': 'IG', 'id': 127585, ...","[{'score': 3, 'team_id': 127585}, {'score': 1,...",False,False,5,"{'acronym': 'IG', 'id': 127585, 'image_url': '...",127585.0,2021-03-07T13:30:41Z,2021-03-07T17:08:57Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5602


Es idéntico al Dataframe que habiamos elaborado en el postwork anterior, es notable que tiene un número de registros mayor, seguramente PandaScore recolecto más datos. Antes de comenzar a tratar los datos, nos gustaría modificar un poco la información que mostramos aprovechando 3 campos que dejamos en forma de lista:

*   _opponents_
*   _results_
*   _winner_

En oponentes tenemos los equipos que se enfrentaron, en results tenemos el puntaje de cada uno y en winner tenemos al equipo ganador. Proponemos entonces crear campos que muestren:

* winner_team
* winner_score
* loser_team
* loser_score

Para ello aprovecharemos las propiedades de listas y los aplicaremos a funciones:

In [424]:
def winner_team(serie):
    if serie == None:
        return 'Undefined'
    else: return serie['name']

La función anterior devuelve el elemento ['name'] de la columna de listas _'winner'_ para aplicarla a toda la columna nos apoyamos del método map():

In [425]:
lol_matches['winner_team'] = lol_matches['winner'].map(winner_team) #Recordemos que algunos enfrentamientos terminaron en empate

Para conocer a los equipos perdedores desglozamos el campo '_opponents_' para que conocer a los equipos y comparamos con el equipo ganador previamente obtenido:

In [426]:
def loser_team(df):
    losers = []
    for i in range(0,len(df)):
        ganador = df['winner_team'][i]
        equipo = df['opponents'][i][0]['opponent']['name']
        if ganador == 'Undefined':
           losers.append('Undefined')
        elif equipo == ganador:
           losers.append(df['opponents'][i][1]['opponent']['name'])  
        elif equipo != ganador: 
           losers.append(df['opponents'][i][0]['opponent']['name'] )
    return pd.Series(losers,index=df.index)

Ya que la función nos devuelve una serie de Pandas podemos añadir la nueva columna:

In [427]:
lol_matches['loser_team'] = loser_team(lol_matches)

Antes de trabajar con las puntuaciones de los equipos, debemos realiar algunos cambios, previamente habiamos llegado al consenso de que en _winner id_ y _winner_ podrían existir valores _None_ debido a que había partidos con empate. Sin embardo para poder extraer los puntajes, es necesario cambiar los _Nones_ para así poder transformar el campo _winner id_ de un tipo de dato float a int y posteriormente realizar la extracción. Primero veamos si existe un _winner id_ que sea igual a cero para saber si podemos convertir los valores NaN en 0: 

In [428]:
lol_matches[ lol_matches['winner_id'] == 0]

,match_name,opponents,results,forfeit,draw,number_of_games,winner,winner_id,begin_date,end_date,rescheduled,official_stream_channel_url,league_id,serie_id,tournament_id,winner_team,loser_team


No hay resultados, podemos transformar con seguridad los _None_ a 0, además corregiremos el campo '_official stream channel url_': 

In [429]:
lol_matches['winner_id'] = lol_matches['winner_id'].fillna(0)
lol_matches['official_stream_channel_url'] = lol_matches['official_stream_channel_url'].fillna('Not Provided')

Veamos si se realizó correctamente el cambio:

In [430]:
lol_matches['winner_id'].isna().sum()
lol_matches['official_stream_channel_url'].isna().sum()

0

!Excelente¡ Ahora transformemos la columna a enteros:

In [431]:
lol_matches['winner_id'] = lol_matches['winner_id'].astype(int)
lol_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18116 entries, 0 to 18115
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   match_name                   18116 non-null  object
 1   opponents                    18116 non-null  object
 2   results                      18116 non-null  object
 3   forfeit                      18116 non-null  bool  
 4   draw                         18116 non-null  bool  
 5   number_of_games              18116 non-null  int64 
 6   winner                       17718 non-null  object
 7   winner_id                    18116 non-null  int64 
 8   begin_date                   18116 non-null  object
 9   end_date                     18116 non-null  object
 10  rescheduled                  18116 non-null  bool  
 11  official_stream_channel_url  18116 non-null  object
 12  league_id                    18116 non-null  int64 
 13  serie_id                     18

Ahora podemos crear nuestra función:

In [432]:
def team_scores(df):
    losers_score = []
    winners_score = []
    for i in range(0,len(df)):
        ganador_id = df['winner_id'][i]
        equipo1_id = df['results'][i][0].get('team_id')
        equipo2_id = df['results'][i][1].get('team_id')
        if ganador_id == 0:
            winner_score = 0
            loser_score = 0
        elif equipo1_id == ganador_id:
            winner_score = df['results'][i][0].get('score')
            loser_score = df['results'][i][1].get('score')
        else:
            winner_score = df['results'][i][1].get('score')
            loser_score = df['results'][i][0].get('score')
        winners_score.append(winner_score)    
        losers_score.append(loser_score)
    return [pd.Series(winners_score,index=df.index), pd.Series(losers_score,index=df.index) ]

In [433]:
[winner_score,loser_score] = team_scores(lol_matches)
lol_matches['winner_score'] = winner_score
lol_matches['loser_score'] = loser_score
lol_matches

,match_name,opponents,results,forfeit,draw,number_of_games,winner,winner_id,begin_date,end_date,rescheduled,official_stream_channel_url,league_id,serie_id,tournament_id,winner_team,loser_team,winner_score,loser_score
0,ALF vs BJD,"[{'opponent': {'acronym': 'ALF', 'id': 124408,...","[{'score': 0, 'team_id': 124408}, {'score': 1,...",False,False,1,"{'acronym': 'BJD', 'id': 126861, 'image_url': ...",126861,2021-03-06T09:54:42Z,2021-03-06T10:28:54Z,False,https://www.twitch.tv/lolpacific,4288,3337,5483,Berjaya Dragons,Alpha Esports,1,0
1,Lower Bracket Round 2 Match 1: CGG vs GGE,"[{'opponent': {'acronym': 'CGG', 'id': 127494,...","[{'score': 1, 'team_id': 127494}, {'score': 3,...",False,False,5,"{'acronym': 'GGE', 'id': 128405, 'image_url': ...",128405,2021-03-08T18:07:48Z,2021-03-08T21:37:26Z,False,https://www.twitch.tv/pg_esports,4405,3291,5579,GG&Esports,Cyberground Gaming,3,1
2,Round 1 match 1: KVM vs ION,"[{'opponent': {'acronym': 'KVM', 'id': 127475,...","[{'score': 3, 'team_id': 127475}, {'score': 0,...",False,False,5,"{'acronym': 'KVM', 'id': 127475, 'image_url': ...",127475,2021-03-08T18:04:03Z,2021-03-08T20:46:01Z,False,https://www.twitch.tv/esportsprimelol,4401,3248,5609,KV Mechelen Esports,Ion Squad,3,0
3,Winners bracket round 1 match 2: ESB vs SIN,"[{'opponent': {'acronym': 'ESB', 'id': 3344, '...","[{'score': 2, 'team_id': 3344}, {'score': 3, '...",False,False,5,"{'acronym': 'SIN', 'id': 127586, 'image_url': ...",127586,2021-03-07T17:53:43Z,2021-03-07T22:22:51Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5602,SINNERS Esports,eSuba,3,2
4,Winners bracket round 1 match 1: IG vs VIK,"[{'opponent': {'acronym': 'IG', 'id': 127585, ...","[{'score': 3, 'team_id': 127585}, {'score': 1,...",False,False,5,"{'acronym': 'IG', 'id': 127585, 'image_url': '...",127585,2021-03-07T13:30:41Z,2021-03-07T17:08:57Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5602,Inside Games,Vikingekrig Esports,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18111,DIG vs C9,"[{'opponent': {'acronym': 'DIG', 'id': 384, 'i...","[{'score': 1, 'team_id': 384}, {'score': 0, 't...",False,False,1,"{'acronym': 'DIG', 'id': 384, 'image_url': 'ht...",384,2015-05-31T20:09:51Z,2015-05-31T20:43:55Z,False,Not Provided,289,31,1,Dignitas,Cloud9,1,0
18112,T8 vs TL,"[{'opponent': {'acronym': 'T8', 'id': 6, 'imag...","[{'score': 0, 'team_id': 6}, {'score': 1, 'tea...",False,False,1,"{'acronym': 'TL', 'id': 390, 'image_url': 'htt...",390,2015-05-31T18:22:58Z,2015-05-31T19:04:18Z,False,Not Provided,289,31,1,Team Liquid,Team 8,1,0
18113,GV vs NME,"[{'opponent': {'acronym': 'GV', 'id': 9, 'imag...","[{'score': 0, 'team_id': 9}, {'score': 1, 'tea...",False,False,1,"{'acronym': 'NME', 'id': 10, 'image_url': 'htt...",10,2015-05-30T23:23:25Z,2015-05-31T00:06:10Z,False,Not Provided,289,31,1,Enemy,Gravity,1,0
18114,T8 vs TIP,"[{'opponent': {'acronym': 'T8', 'id': 6, 'imag...","[{'score': 0, 'team_id': 6}, {'score': 1, 'tea...",False,False,1,"{'acronym': 'TIP', 'id': 385, 'image_url': 'ht...",385,2015-05-30T22:23:32Z,2015-05-30T22:56:32Z,False,Not Provided,289,31,1,Impulse,Team 8,1,0


Ahora que extrajimos la información más relevante de los campos con listas, podemos retirarlos:

In [466]:
lol_matches2 = lol_matches
campos_no_deseados = ['results','winner','opponents','winner_id']
lol_matches2 = lol_matches2.drop(columns=campos_no_deseados)

In [467]:
lol_matches2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18116 entries, 0 to 18115
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   match_name                   18116 non-null  object
 1   forfeit                      18116 non-null  bool  
 2   draw                         18116 non-null  bool  
 3   number_of_games              18116 non-null  int64 
 4   begin_date                   18116 non-null  object
 5   end_date                     18116 non-null  object
 6   rescheduled                  18116 non-null  bool  
 7   official_stream_channel_url  18116 non-null  object
 8   league_id                    18116 non-null  int64 
 9   serie_id                     18116 non-null  int64 
 10  tournament_id                18116 non-null  int64 
 11  winner_team                  18116 non-null  object
 12  loser_team                   18116 non-null  object
 13  winner_score                 18

Nuestro siguiente objetivo es refinar los tipos de datos de los campos para las fechas:

In [468]:
from datetime import *
lol_matches2['begin_date'] = lol_matches2['begin_date'].apply(datetime.strptime, args=('%Y-%m-%dT%H:%M:%SZ',))
lol_matches2['end_date'] = lol_matches2['end_date'].apply(datetime.strptime, args=('%Y-%m-%dT%H:%M:%SZ',))


Ya que la API guarda los datos en una forma algo peculiar nos apoyamos del paquete datetime y convertimos con ayuda de apply. Finalmente tendremos un Dataframe libre de valores nulos, con campos nombrados coherentementes (en inglés) y que ofrece la posibilidad de trabajar con otros datos de la API al conservar los identificadores:

In [463]:
lol_matches2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18116 entries, 0 to 18115
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   match_name                   18116 non-null  object        
 1   forfeit                      18116 non-null  bool          
 2   draw                         18116 non-null  bool          
 3   number_of_games              18116 non-null  int64         
 4   begin_date                   18116 non-null  datetime64[ns]
 5   end_date                     18116 non-null  datetime64[ns]
 6   rescheduled                  18116 non-null  bool          
 7   official_stream_channel_url  18116 non-null  object        
 8   league_id                    18116 non-null  int64         
 9   serie_id                     18116 non-null  int64         
 10  tournament_id                18116 non-null  int64         
 11  winner_team                  18116 non-nu

In [476]:
lol_matches2.head()

,match_name,forfeit,draw,number_of_games,begin_date,end_date,rescheduled,official_stream_channel_url,league_id,serie_id,tournament_id,winner_team,loser_team,winner_score,loser_score
0,ALF vs BJD,False,False,1,2021-03-06 09:54:42,2021-03-06 10:28:54,False,https://www.twitch.tv/lolpacific,4288,3337,5483,Berjaya Dragons,Alpha Esports,1,0
1,Lower Bracket Round 2 Match 1: CGG vs GGE,False,False,5,2021-03-08 18:07:48,2021-03-08 21:37:26,False,https://www.twitch.tv/pg_esports,4405,3291,5579,GG&Esports,Cyberground Gaming,3,1
2,Round 1 match 1: KVM vs ION,False,False,5,2021-03-08 18:04:03,2021-03-08 20:46:01,False,https://www.twitch.tv/esportsprimelol,4401,3248,5609,KV Mechelen Esports,Ion Squad,3,0
3,Winners bracket round 1 match 2: ESB vs SIN,False,False,5,2021-03-07 17:53:43,2021-03-07 22:22:51,False,https://www.twitch.tv/hitpointcz,4433,3314,5602,SINNERS Esports,eSuba,3,2
4,Winners bracket round 1 match 1: IG vs VIK,False,False,5,2021-03-07 13:30:41,2021-03-07 17:08:57,False,https://www.twitch.tv/hitpointcz,4433,3314,5602,Inside Games,Vikingekrig Esports,3,1


Antes de guardar nuestro Dataframe, ordenemos por fecha de forma que lo primero que veamos sean los enfrentamientos más antiguos:

In [478]:
lol_matches2 = lol_matches2.sort_values('begin_date', ascending=True)

Necesitamos resetear el índice:

In [479]:
lol_matches2 = lol_matches2.reset_index(drop=True)
lol_matches2

,match_name,forfeit,draw,number_of_games,begin_date,end_date,rescheduled,official_stream_channel_url,league_id,serie_id,tournament_id,winner_team,loser_team,winner_score,loser_score
0,Qf 2,False,False,3,2014-06-01 14:00:00,2014-06-01 15:35:19,False,Not Provided,291,1352,696,Team LoLPro,Area of Effect eSports,2,0
1,Qf 3,False,False,3,2014-06-07 14:00:00,2014-06-07 15:40:14,False,Not Provided,291,1352,696,Team Coast,Team Green Forest,2,0
2,Sf1,False,False,3,2014-06-14 14:00:00,2014-06-14 15:30:36,False,Not Provided,291,1352,696,Team LoLPro,Frank Fang Gaming,2,0
3,Sf2,False,False,3,2014-06-15 14:00:00,2014-06-15 16:35:43,False,Not Provided,291,1352,696,Team Coast,Curse Academy,2,1
4,3rd/4th,False,False,3,2014-06-16 14:00:00,2014-06-16 15:43:55,False,Not Provided,291,1352,696,Curse Academy,Frank Fang Gaming,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18111,Week 7: STR vs x25 Esports,False,False,1,2021-03-08 21:01:47,2021-03-08 21:35:43,False,https://www.twitch.tv/fortuna,4426,3351,5514,x25 Esports,Split Raiders,1,0
18112,KRM vs WD,False,False,1,2021-03-08 21:06:24,2021-03-08 21:42:41,False,https://www.twitch.tv/InygonTV1,4407,3301,5411,White Dragons,Karma Clan Esports,1,0
18113,MRS vs UCAM,False,False,1,2021-03-08 21:09:58,2021-03-08 21:58:38,True,http://www.twitch.tv/lvpes,4213,3255,5346,UCAM Esports Club,Movistar Riders,1,0
18114,E5 vs GTZ,False,False,1,2021-03-08 22:07:54,2021-03-08 22:44:31,False,https://www.twitch.tv/InygonTV1,4407,3301,5411,GTZ Bulls,EFIVE Esports,1,0


Finalmente, guardamos el Dataframe:

In [480]:
lol_matches2.to_csv('lol_matches2.csv')